<a href="https://colab.research.google.com/github/PhamBaVuong/PhamBaVuong_DATN/blob/main/MachineTranslation_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import collections

import helper
import numpy as np


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
import os


def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()
 
    return data.split('\n')

In [4]:
# Load tên đường
ten_duong = load_data('/content/drive/MyDrive/DATN/Quan_1_TenDuong.csv')
# Load Tọa độ
toa_do = load_data('/content/drive/MyDrive/DATN/Quan_1_ToaDo.csv')

print('Dataset Loaded')

Dataset Loaded


In [5]:
for sample_i in range(2):#len(ten_duong) - 1
    print('Tên đường {}:  {}'.format(sample_i + 1, ten_duong[sample_i]))
    print('Tọa độ {}:  {}'.format(sample_i + 1, toa_do[sample_i]))

Tên đường 1:  ﻿Alexandre de Rhodes
Tọa độ 1:  ﻿"10.779634866903798, 106.69683645007238"
Tên đường 2:  Bà Lê Chân
Tọa độ 2:  "10.790615971684014, 106.68954129640592"


In [6]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
 
    return x_tk.texts_to_sequences(x), x_tk

#tests.test_tokenize(tokenize)
 
# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [7]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')
#tests.test_pad(pad)
 
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [8]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
 
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
 
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
 
    return preprocess_x, preprocess_y, x_tk, y_tk
 
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(ten_duong, toa_do)
     
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)
 
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 4
Max French sentence length: 5
English vocabulary size: 221
French vocabulary size: 281


In [9]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
 
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [10]:
french_vocab_size

281

In [11]:
from keras.layers import GRU, Input, Dense, TimeDistributed, Dropout, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape=input_shape[1:], return_sequences=True))
    model.add(Dropout(0.5))
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size+1, activation='softmax'))) 
     
     
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
#tests.test_simple_model(simple_model)
 
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
 
 
 
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=300, epochs=10, validation_split=0.2)
 
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 5.6249 - accuracy: 0.0179 - val_loss: 5.4917 - val_accuracy: 0.2286
Epoch 2/10
1/1 [==============================] - 0s 95ms/step - loss: 5.5379 - accuracy: 0.0768 - val_loss: 5.3745 - val_accuracy: 0.2286
Epoch 3/10
1/1 [==============================] - 0s 90ms/step - loss: 5.4209 - accuracy: 0.1214 - val_loss: 5.2462 - val_accuracy: 0.2286
Epoch 4/10
1/1 [==============================] - 0s 94ms/step - loss: 5.3127 - accuracy: 0.1714 - val_loss: 5.0998 - val_accuracy: 0.2286
Epoch 5/10
1/1 [==============================] - 0s 87ms/step - loss: 5.1535 - accuracy: 0.2214 - val_loss: 4.9315 - val_accuracy: 0.2286
Epoch 6/10
1/1 [==============================] - 0s 109ms/step - loss: 4.9595 - accuracy: 0.2357 - val_loss: 4.7424 - val_accuracy: 0.2286
Epoch 7/10
1/1 [==============================] - 0s 93ms/step - loss: 4.8114 - accuracy: 0.2304 - val_loss: 4.5440 - val_accuracy: 0.2286
Epoch 8/10
1/1 [============

In [12]:
simple_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 5, 128)            50304     
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 gru_1 (GRU)                 (None, 5, 128)            99072     
                                                                 
 dropout_1 (Dropout)         (None, 5, 128)            0         
                                                                 
 time_distributed (TimeDistr  (None, 5, 256)           33024     
 ibuted)                                                         
                                                                 
 dropout_2 (Dropout)         (None, 5, 256)            0         
                                                        

In [13]:
import os
import glob
 
if not os.path.exists("models"):
    os.makedirs("models")
 
from keras.models import load_model
cache_dir = os.path.join("models")
model_file = "rnn_model.h5"
simple_rnn_model.save(os.path.join(cache_dir, model_file))
 
os.path.join(cache_dir, model_file)

'models/rnn_model.h5'

In [14]:
from keras.layers.embeddings import Embedding
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate= 0.001
    model = Sequential()
    model.add(Embedding(english_vocab_size, 100, input_length=input_shape[1], input_shape=input_shape[1:]))
     
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(GRU(128,  return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size+1, activation='softmax'))) 
     
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
     
  
    return model
#tests.test_embed_model(embed_model)
 
 
#  Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
 
#  Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

In [15]:
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=300, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 5.6438 - accuracy: 0.0036 - val_loss: 5.6340 - val_accuracy: 0.3000
Epoch 2/10
1/1 [==============================] - 0s 95ms/step - loss: 5.6338 - accuracy: 0.1143 - val_loss: 5.6218 - val_accuracy: 0.2714
Epoch 3/10
1/1 [==============================] - 0s 89ms/step - loss: 5.6221 - accuracy: 0.2429 - val_loss: 5.6067 - val_accuracy: 0.2286
Epoch 4/10
1/1 [==============================] - 0s 91ms/step - loss: 5.6077 - accuracy: 0.2589 - val_loss: 5.5875 - val_accuracy: 0.2286
Epoch 5/10
1/1 [==============================] - 0s 101ms/step - loss: 5.5857 - accuracy: 0.2536 - val_loss: 5.5630 - val_accuracy: 0.2286
Epoch 6/10
1/1 [==============================] - 0s 87ms/step - loss: 5.5598 - accuracy: 0.2393 - val_loss: 5.5317 - val_accuracy: 0.2286
Epoch 7/10
1/1 [==============================] - 0s 92ms/step - loss: 5.5278 - accuracy: 0.2393 - val_loss: 5.4914 - val_accuracy: 0.2286
Epoch 8/10
1/1 [============

In [16]:
embed_rnn_model.summary()
model_file = "embed_model.h5"
embed_rnn_model.save(os.path.join(cache_dir, model_file))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            22200     
                                                                 
 gru_2 (GRU)                 (None, 5, 128)            88320     
                                                                 
 dropout_3 (Dropout)         (None, 5, 128)            0         
                                                                 
 gru_3 (GRU)                 (None, 5, 128)            99072     
                                                                 
 dropout_4 (Dropout)         (None, 5, 128)            0         
                                                                 
 time_distributed_2 (TimeDis  (None, 5, 256)           33024     
 tributed)                                                       
                                                      

In [17]:
print(ten_duong[:1])
print(toa_do[:1])

['\ufeffAlexandre de Rhodes']
['\ufeff"10.779634866903798, 106.69683645007238"']


In [18]:
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

<PAD> <PAD> <PAD> <PAD> <PAD>


In [20]:
from keras.layers import Bidirectional
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
     
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(Dropout(0.5))
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 
     
     
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
     
     
    return model
     
# tests.test_bd_model(bd_model)
 
 
# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
 
# Train and Print prediction(s)
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

In [21]:
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=300, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 5.6922 - accuracy: 0.0036 - val_loss: 5.3388 - val_accuracy: 0.4071
Epoch 2/10
1/1 [==============================] - 0s 142ms/step - loss: 5.4467 - accuracy: 0.0339 - val_loss: 5.0710 - val_accuracy: 0.5143
Epoch 3/10
1/1 [==============================] - 0s 128ms/step - loss: 5.1916 - accuracy: 0.2000 - val_loss: 4.7857 - val_accuracy: 0.5643
Epoch 4/10
1/1 [==============================] - 0s 131ms/step - loss: 4.9234 - accuracy: 0.3625 - val_loss: 4.4692 - val_accuracy: 0.5857
Epoch 5/10
1/1 [==============================] - 0s 125ms/step - loss: 4.6739 - accuracy: 0.4179 - val_loss: 4.1328 - val_accuracy: 0.5929
Epoch 6/10
1/1 [==============================] - 0s 128ms/step - loss: 4.3355 - accuracy: 0.4518 - val_loss: 3.8231 - val_accuracy: 0.5857
Epoch 7/10
1/1 [==============================] - 0s 133ms/step - loss: 4.0569 - accuracy: 0.4429 - val_loss: 3.6168 - val_accuracy: 0.5857
Epoch 8/10
1/1 [=====

In [22]:
bd_rnn_model.summary()
model_file = "bd_rnn_model.h5"
bd_rnn_model.save(os.path.join(cache_dir, model_file))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 5, 256)           100608    
 l)                                                              
                                                                 
 dropout_6 (Dropout)         (None, 5, 256)            0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 5, 256)           296448    
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 5, 256)            0         
                                                                 
 time_distributed_4 (TimeDis  (None, 5, 256)           65792     
 tributed)                                                       
                                                      

In [24]:
from keras.layers import RepeatVector
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    model = Sequential()
     
    #Encoder
    inputs = Input(shape=input_shape[1:])
    gru = GRU(output_sequence_length)(inputs)
    e_out = Dense(128, activation='relu')(gru)
     
    #Decoder
    d_input = RepeatVector(output_sequence_length)(e_out)
    d_gru = GRU(128, return_sequences=True)(d_input)
    layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    final = layer(d_gru)
 
    #Create Model from parameters defined above
    model = Model(inputs=inputs, outputs=final)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
     
     
    return model
     
     
     
# tests.test_encdec_model(encdec_model)
 
# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
 
# Train and Print prediction(s)
ed_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
 
ed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=300, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 5.6303 - accuracy: 0.0036 - val_loss: 5.6072 - val_accuracy: 0.0643
Epoch 2/10
1/1 [==============================] - 0s 70ms/step - loss: 5.6103 - accuracy: 0.0643 - val_loss: 5.5865 - val_accuracy: 0.2214
Epoch 3/10
1/1 [==============================] - 0s 69ms/step - loss: 5.5898 - accuracy: 0.2036 - val_loss: 5.5647 - val_accuracy: 0.4214
Epoch 4/10
1/1 [==============================] - 0s 82ms/step - loss: 5.5683 - accuracy: 0.3839 - val_loss: 5.5410 - val_accuracy: 0.3857
Epoch 5/10
1/1 [==============================] - 0s 65ms/step - loss: 5.5450 - accuracy: 0.3357 - val_loss: 5.5148 - val_accuracy: 0.2286
Epoch 6/10
1/1 [==============================] - 0s 89ms/step - loss: 5.5191 - accuracy: 0.2018 - val_loss: 5.4851 - val_accuracy: 0.2286
Epoch 7/10
1/1 [==============================] - 0s 69ms/step - loss: 5.4900 - accuracy: 0.1982 - val_loss: 5.4513 - val_accuracy: 0.2286
Epoch 8/10
1/1 [=============

In [25]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.001
    inputs = Input(shape=input_shape[1:])
    emb = Embedding(english_vocab_size, 100)(inputs)
    gru = Bidirectional(GRU(128, dropout=0.5))(emb)
    final_enc = Dense(256, activation='relu')(gru)
     
    dec1 = RepeatVector(output_sequence_length)(final_enc)
    decgru = Bidirectional(LSTM(512, dropout=0.2, return_sequences=True))(dec1)
    layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    final = layer(decgru)
     
     
    model = Model(inputs=inputs, outputs=final)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
     
  
    return model
#tests.test_model_final(model_final)

In [32]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(
        x.shape,
        y.shape[1],
        len(x_tk.word_index)+1,
        len(y_tk.word_index)+1)
    print (model.summary())
    model.fit(x, y, batch_size=300, epochs=30, validation_split=0.2)
 
     
     
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'
 
    sentence = 'Alexandre de Rhodes'
    sentence = [x_tk.word_index[word] for word in sentence.split(' ')]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))
 
    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
 
 
final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 4)]               0         
                                                                 
 embedding_7 (Embedding)     (None, 4, 100)            22200     
                                                                 
 bidirectional_14 (Bidirecti  (None, 256)              176640    
 onal)                                                           
                                                                 
 dense_20 (Dense)            (None, 256)               65792     
                                                                 
 repeat_vector_7 (RepeatVect  (None, 5, 256)           0         
 or)                                                             
                                                                 
 bidirectional_15 (Bidirecti  (None, 5, 1024)          3149

KeyError: ignored